# Auto Gen Tutorial
Note book written by John Adeojo
Founder, and Chief Data Scientist at [Data-centric Solutions](https://www.data-centric-solutions.com/).

---
# License

This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

## How to Credit

If you use this work or adapt it, please credit the author and the company as follows:

"Auto Gen Tutorial: Open Domain Question Answering with Wikipedia" by John Adeojo from Data-Centric Solutions, used under CC BY 4.0 / Desaturated from original

## Example Citation

In academic or professional contexts, you can cite this work as follows:

Adeojo, John. "Auto Gen Tutorial: Open Domain Question Answering with Wikipedia." Data-Centric Solutions. 13/11/2023


## 0. Utility Functions

In [14]:
import yaml
import os

def get_apikey():
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The OpenAI API key.
    """
    
    # Construct the full path to the configuration file
    script_dir = "G:/My Drive/Data-Centric Solutions/07. Blog Posts/AutoGen 2 - Flights"
    file_path = os.path.join(script_dir, "amadeus_api.yml")

    with open(file_path, 'r') as yamlfile:
        loaded_yamlfile = yaml.safe_load(yamlfile)
        api_key = loaded_yamlfile['api_key']
        api_secret = loaded_yamlfile['api_secret']
        dbname = loaded_yamlfile['host']
        
    return api_key, api_secret

api_key, api_secret = get_apikey()
# print(f"api key:{api_key}, api secret:{api_secret}")

## 1. Define functions

Define functions for requesting data from [Amadeus flights](https://developers.amadeus.com/self-service/category/flights/api-doc/flight-offers-search/api-reference) API.

In [15]:
import logging
from amadeus import Client, ResponseError
from datetime import datetime


# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('amadeus')

def search_flight_offers_with_amadeus_client(api_key, api_secret, originLocationCode, destinationLocationCode, departureDate, 
                                             returnDate=None, adults=1, children=None, infants=None, travelClass=None, 
                                             includedAirlineCodes=None, excludedAirlineCodes=None, nonStop="false", 
                                             currencyCode=None, maxPrice=None, max=5):
    """
    Search for flight offers using Amadeus Python SDK.
    """

    # Initialize Amadeus client
    amadeus = Client(
        client_id=api_key,
        client_secret=api_secret
    )

    # Set up the parameters for the request
    params = {
        'originLocationCode': originLocationCode,
        'destinationLocationCode': destinationLocationCode,
        'departureDate': departureDate,
        'adults': adults,
        'returnDate': returnDate,
        'children': children,
        'infants': infants,
        'travelClass': travelClass,
        'includedAirlineCodes': includedAirlineCodes,
        'excludedAirlineCodes': excludedAirlineCodes,
        'nonStop': nonStop,
        'currencyCode': currencyCode,
        'maxPrice': maxPrice,
        'max': max
    }

    # Remove None values
    params = {k: v for k, v in params.items() if v is not None}
    logger.debug("Request parameters: %s", params)

    # Make the request using Amadeus client
    try:
        response = amadeus.shopping.flight_offers_search.get(**params)
        logger.debug("Response data: %s", response.data)
        return response.data
    except ResponseError as error:
        logger.error(f"An error occurred: {error}")
        logger.error(f"Error code: {error.code}")
        logger.error(f"Error message: {error.description}")
        if hasattr(error, 'response'):
            logger.error("Full error response: %s", error.response.body)  # Log full error response
        return None


def get_todays_date():
    """Returns today's date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")

# Example usage
today_date = get_todays_date()
print(today_date)

# Example usage
api_key = api_key
api_secret = api_secret
offers = search_flight_offers_with_amadeus_client(api_key, api_secret, 'SYD', 'BKK', '2023-12-20', adults=1)

2023-11-14
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}
Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}


DEBUG:amadeus:Request parameters: {'originLocationCode': 'SYD', 'destinationLocationCode': 'BKK', 'departureDate': '2023-12-20', 'adults': 1, 'nonStop': 'false', 'max': 5}


Response data: [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-12-20', 'lastTicketingDateTime': '2023-12-20', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT26H55M', 'segments': [{'departure': {'iataCode': 'SYD', 'terminal': '1', 'at': '2023-12-20T12:20:00'}, 'arrival': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-20T18:40:00'}, 'carrierCode': 'MF', 'number': '802', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT9H20M', 'id': '8', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-21T08:50:00'}, 'arrival': {'iataCode': 'BKK', 'at': '2023-12-21T11:15:00'}, 'carrierCode': 'MF', 'number': '853', 'aircraft': {'code': '738'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT3H25M', 'id': '9', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR

DEBUG:amadeus:Response data: [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-12-20', 'lastTicketingDateTime': '2023-12-20', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT26H55M', 'segments': [{'departure': {'iataCode': 'SYD', 'terminal': '1', 'at': '2023-12-20T12:20:00'}, 'arrival': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-20T18:40:00'}, 'carrierCode': 'MF', 'number': '802', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT9H20M', 'id': '8', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-21T08:50:00'}, 'arrival': {'iataCode': 'BKK', 'at': '2023-12-21T11:15:00'}, 'carrierCode': 'MF', 'number': '853', 'aircraft': {'code': '738'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT3H25M', 'id': '9', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'c

In [16]:
offers

[{'type': 'flight-offer',
  'id': '1',
  'source': 'GDS',
  'instantTicketingRequired': False,
  'nonHomogeneous': False,
  'oneWay': False,
  'lastTicketingDate': '2023-12-20',
  'lastTicketingDateTime': '2023-12-20',
  'numberOfBookableSeats': 9,
  'itineraries': [{'duration': 'PT26H55M',
    'segments': [{'departure': {'iataCode': 'SYD',
       'terminal': '1',
       'at': '2023-12-20T12:20:00'},
      'arrival': {'iataCode': 'XMN',
       'terminal': '3',
       'at': '2023-12-20T18:40:00'},
      'carrierCode': 'MF',
      'number': '802',
      'aircraft': {'code': '789'},
      'operating': {'carrierCode': 'MF'},
      'duration': 'PT9H20M',
      'id': '8',
      'numberOfStops': 0,
      'blacklistedInEU': False},
     {'departure': {'iataCode': 'XMN',
       'terminal': '3',
       'at': '2023-12-21T08:50:00'},
      'arrival': {'iataCode': 'BKK', 'at': '2023-12-21T11:15:00'},
      'carrierCode': 'MF',
      'number': '853',
      'aircraft': {'code': '738'},
      'operati

In [17]:
# Import the function from the update_tables module
from update_tabs import process_amadeus_data, make_engine, apis_configs

_, _, host, password = apis_configs()
database = "flights_data"
username = "john-adeojo"
port = "5432"

# Use the function
engine = make_engine(username, password, host, database)
json_data = offers  # Your JSON data from the Amadeus API
process_amadeus_data(json_data, engine)

[{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-12-20', 'lastTicketingDateTime': '2023-12-20', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT26H55M', 'segments': [{'departure': {'iataCode': 'SYD', 'terminal': '1', 'at': '2023-12-20T12:20:00'}, 'arrival': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-20T18:40:00'}, 'carrierCode': 'MF', 'number': '802', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT9H20M', 'id': '8', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'XMN', 'terminal': '3', 'at': '2023-12-21T08:50:00'}, 'arrival': {'iataCode': 'BKK', 'at': '2023-12-21T11:15:00'}, 'carrierCode': 'MF', 'number': '853', 'aircraft': {'code': '738'}, 'operating': {'carrierCode': 'MF'}, 'duration': 'PT3H25M', 'id': '9', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '30

In [18]:
from update_tables import process_amadeus_data, create_engine, apis_configs